In [1]:
import os
import logging
import networkx as nx
import numpy as np
#import seaborn as sns
import random
import math
from scipy.stats import truncnorm
import time
import copy
import matplotlib.pyplot as plt
from random import sample
import itertools
from collections import Counter
from tqdm import tqdm
data_name = "auth"
NCCPATH = '../data/LANL/TCG_LANL.txt'
SEED = 1

MasterGraph = nx.DiGraph()
MGedges = set()
wei = {}
time_link = {}
with open(NCCPATH, 'r', encoding='utf-8') as fi:
    fi.readline()
    total_lines = sum(1 for line in fi)
    fi.seek(0)
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        if((l[3], l[6])in MGedges):
            MasterGraph.edges[l[3],l[6]]['weight'] +=1
            MasterGraph.edges[l[3],l[6]]['time'] = l[0]
            time_link[l[3],l[6]].add(int(l[0]))
        else:
            MGedges.add((l[3], l[6]))
            MasterGraph.add_edge(l[3], l[6], time=l[0],weight=1)
            time_link[l[3], l[6]] = set()
            time_link[l[3], l[6]].add(int(l[0]))

In [2]:
num_nodes = MasterGraph.number_of_nodes()
num_edges = MasterGraph.number_of_edges()


print("Total nodes:", num_nodes)
print("Total edges:", num_edges)

Total nodes: 15343
Total edges: 163371


In [3]:
sub_Graph = nx.DiGraph()
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team, 'r').readlines())
red_edge = []
for red in red_file:
    l = red.strip().split(',')
    if ([l[3], l[4]] not in red_edge):
        red_edge.append([l[3], l[4]])
red_edge.remove([' source', 'target'])
red_graph = nx.DiGraph()
for e in red_edge:
    red_graph.add_edge(e[0],e[1])

# Potential attack path discovery guided by target

In [4]:
zero_id = set([key for key, value in dict(MasterGraph.in_degree()).items() if value == 0] )
tar ={'C2189','C528', 'C2388','C1065'}
sta = list(zero_id)
print('number of target and source nodes',len(tar),len(sta))

number of target and source nodes 4 7831


In [5]:
m = 1
sz = math.ceil(len(sta)/m)
FS = {}
prohibit = {}  
for i in range(0,m):
    prohibit[i] = set()
    FS[i] = set()
for i in range(0,m):
    if(len(sta)<sz):
        FS[i] =sta
    else:
        FS[i] = np.random.choice(sta, size=sz, replace=False)
    sta = list(set(sta) - set(FS[i]))
    print(len(sta))
path_list = {}
path_at = path_list.fromkeys(list(MasterGraph.nodes()))
path_unat = path_list.fromkeys(list(MasterGraph.nodes()))
for key in path_at:
    path_at[key] = []
    path_unat[key] = set()

0


In [6]:
def update(source,pro_node):
    next_nodes = []
    for node in source:
        neigh = list(nx.neighbors(MasterGraph, node))
        access = list(set(neigh)-pro_node)
        for key in access:
            path_unat[key].add(node)
        next_nodes.extend(access)
    source = copy.deepcopy(set(next_nodes))
    return source,set(next_nodes)
iter = 0
redundancy = set()
merged_values = set(itertools.chain(*FS.values()))
while (len(merged_values)>0):
    print("iter",iter)
    for i in range(0, m):
        fs_temph,pro = update( FS[i],prohibit[i])
        prohibit[i].update(pro)
        prohibit[i] = prohibit[i]-set(tar)
        FS[i] = fs_temph
    merged_values = set(itertools.chain(*FS.values()))-tar
    iter+=1

iter 0
iter 1
iter 2
iter 3
iter 4


In [7]:
def Computepath(key, keys, reNei, edge):
#     print(reNei[key])
    if (key not in keys) and (reNei[key] != []):
        keys.add(key)
        for x in reNei[key]:
            edge.append([key, x])
            Computepath(x, keys, reNei, edge)
    return edge


for key in tar:
    edge = []
    keys = set()
    path_at[key] = Computepath(key, keys, path_unat, edge)

edgelist = []
target = []
for k in path_at:
    if(path_at[k]==[]):
        continue
    edgelist.extend(path_at[k])
keynode = set()
for n in edgelist:
    keynode.add(n[0])
    keynode.add(n[1])
print(len(edgelist), len(keynode))


141010 14320


# Potential atack path discovery based on rare events.

In [8]:
#add edge frequency
sum_wei = {}
lamda = {}
for e in MasterGraph.edges():
    day_number = (max(time_link[e])-min(time_link[e]))/86400
    lamda[e] = MasterGraph.edges[(e)]['weight']/(day_number+1)
    if(e[0] in sum_wei):
        sum_wei[e[0]] += lamda[e]
    else:
        sum_wei[e[0]] = lamda[e]

#     print(MasterGraph.edges[e]['oweight'])



In [9]:
for e in MasterGraph.edges():
    MasterGraph.edges[e]['oweight']=math.ceil((lamda[e]*lamda[e]) / sum_wei[e[0]])

In [10]:
right_edge = []
MG = nx.DiGraph()
for e in edgelist:
    right_edge.append([e[1], e[0]])
    MG.add_edge(e[1], e[0],time = int(MasterGraph.edges[e[1], e[0]]['time']),weight = MasterGraph.edges[e[1], e[0]]['oweight'])

km = 0
for e in right_edge:
    if ('C17693' in e or 'C18025' in e or 'C19932' in e or 'C22409' in e):
        km += 1
print('num of attack host links', km)

num of attack host links 918


In [11]:
k = 0
for red_e in red_edge:
    if (red_e in right_edge):
        k += 1
print('num of attack links', k)
sta = list(zero_id)
#shortest path at target
all_simple_edges = set()
def path_to_edge(paths):
    e = set()
    for path in paths:
        for l in range(1,len(path)):
            e.add((path[l-1],path[l]))
    return e
source = set([key for key, value in dict(MG.in_degree()).items() if value == 0] ).intersection(sta)

print("number of source at target ",len(source))

num of attack links 274
number of source at target  7744


In [12]:

def compute_rare_path(MG,all_simple_edges):
    current_scan = 0
    for n1 in tar:
        for n2 in source:
            try:
                paths = list(nx.all_shortest_paths(MG,n2,n1,'weight'))
                # paths = [path for path, weight in top_10_weighted_paths(MG,n2,n1)]
                every_simple_edges = path_to_edge(paths)
                all_simple_edges.update(every_simple_edges)
            except nx.NetworkXNoPath:
                    logging.info('There are no path detected in the graph')
                    continue
        current_scan+=1
        # print('current_scan',current_scan)
    return all_simple_edges
all_simple_edges = compute_rare_path(MG,all_simple_edges)
k = 0
num_attnode = set()
for red_e in red_edge:
    if (tuple(red_e) in all_simple_edges):
        k += 1
        num_attnode.add(red_e[0])
        num_attnode.add(red_e[1])
print('num of attack links， nodes',k,len(num_attnode))

all_nodes = set()
all_edges = []
for e in all_simple_edges:
    all_nodes.add(e[0])
    all_nodes.add(e[1])
    all_edges.append([e[0],e[1]])
print("filter edges and nodes",len(all_edges),len(all_nodes))

num of attack links， nodes 185 187
filter edges and nodes 28420 8538


In [13]:
for m in range(5):
    for e in all_simple_edges:
        if e not in MG.edges():
            continue
        MG.remove_edge(e[0],e[1])
    all_simple_edges = compute_rare_path(MG,all_simple_edges)
    # print('number of edges at target',len(all_simple_edges))
    k = 0
    num_attnode = set()
    for red_e in red_edge:
        if (tuple(red_e) in all_simple_edges):
            k += 1
            num_attnode.add(red_e[0])
            num_attnode.add(red_e[1])
    print('num of attack links， nodes',k,len(num_attnode))

    all_nodes = set()
    all_edges = []
    for e in all_simple_edges:
        all_nodes.add(e[0])
        all_nodes.add(e[1])
        all_edges.append([e[0],e[1]])
    print("filter edges and nodes",len(all_edges),len(all_nodes))

num of attack links， nodes 230 233
filter edges and nodes 42842 13075
num of attack links， nodes 238 241
filter edges and nodes 44307 13658
num of attack links， nodes 242 245
filter edges and nodes 44832 13876
num of attack links， nodes 244 247
filter edges and nodes 45093 13990
num of attack links， nodes 245 248
filter edges and nodes 45181 14031


# Potential atack path selection under time constraints

In [14]:
in_edge = {}
for e in MG.edges():
    if (e[1] in in_edge):
        in_edge[e[1]].add(int(MG.edges[e[0], e[1]]['time']))
    else:
        in_edge[e[1]] = set()
        in_edge[e[1]].add(int(MG.edges[e[0], e[1]]['time']))

Min_int = {}
for x in in_edge:
    Min_int[x] = min(in_edge[x])
np.savetxt('../data/LANL/new_edge.txt',all_edges, fmt='%s,%s')
np.savetxt("../data/LANL/new_node.txt", np.array(list(all_nodes)), fmt='%s')

CCPATH = '../data/LANL/new_edge.txt'
TUDCCPATH = '../data/LANL/AIG_LANL.txt'
fCC = open(CCPATH, 'r')
CC = fCC.readlines()
Sort_CC = set(sorted(CC))
with open(TUDCCPATH, 'a') as sf:
    sf.truncate(0)
    sf.close()
i = 0
k = 0
with open(NCCPATH, 'r', encoding='utf-8') as fi:
    fi.readline()
    start = time.time()
    sf = open(TUDCCPATH, 'a')
    while True:
        line = fi.readline()
        i = i + 1
        lline = line.strip().split(',')
        if not line:
            break
        scc = str(lline[3]) + ',' + str(lline[6]) + '\n'
        if (scc in (Sort_CC)):
            if (lline[3] in in_edge):
                if (int(lline[0]) > Min_int[lline[3]]):
                    sf.write(line)
            else:
                sf.write(line)
            k += 1
        if (i % 1000000 == 0):
            print(time.time() - start)
fi.close()
fCC.close()
sf.close()

j = 0
label = np.zeros(k)
sub_Graph = nx.DiGraph()
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team, 'r').readlines())
with open(TUDCCPATH, 'r', encoding='utf-8') as fi:
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
        sub_Graph.add_edge(l[3], l[6])
        if str_line in red_file:
            label[j] = 1
        j += 1
    fi.close()
print("num of attack events",sum(label))

def iter_count(file_name):
    from itertools import (takewhile, repeat)
    buffer = 1024 * 1024
    with open(file_name) as f:
        buf_gen = takewhile(lambda x: x, (f.read(buffer) for _ in repeat(None)))
        return sum(buf.count('\n') for buf in buf_gen)


print("num of all events",iter_count(TUDCCPATH))

1.2185046672821045
2.4362504482269287
3.650578022003174
4.855380296707153
6.0746965408325195
7.280638933181763
8.468815565109253
9.659446477890015
10.84904146194458
12.053351879119873
13.262638568878174
14.461922645568848
15.660815238952637
16.849971532821655
18.040684938430786
19.227465391159058
20.39924454689026
21.569523334503174
22.739723682403564
23.90704607963562
25.073901891708374
26.239107608795166
27.407455682754517
28.56575345993042
29.733853816986084
30.934895992279053
32.14112138748169
33.33604884147644
34.51798701286316
35.72206401824951
36.92845916748047
38.13573431968689
39.3616738319397
40.57926964759827
41.79654026031494
43.01736879348755
num of attack events 569.0
num of all events 4979752
